In [1]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import pandas as pd
import os
import seaborn as sns
import random 
import scipy as sc
import statsmodels.api as sm
import scipy.stats as stats
import cartopy.crs as ccrs
import cartopy as cp
import pygrib
import cfgrib
import stormeunice as eun

sns.set_theme(style="white")
sns.set_style("white")

random.seed(10)

In [2]:
# dir(eun.data.Data

# Description

Identify extreme ensemble members in the operational forecast

Then, analyse these ensemble members in all three experiments (pi, curr, incr) wrt
- minimum pressure
- storm track 

# Import data

In [3]:
directory, experiments, inits, cfpf = eun.data.Data.load_meta()
lat, lon = eun.data.Data.get_latlon()
south_df = eun.data.Data.get_friday_data()

In [4]:
# Create an xarray for ease of use
south_xr = south_df.to_xarray().set_index(index = ("lat", "lon", "time"))  # .expand_dims()

<xarray.Dataset>
Dimensions:     (index: 745416)
Coordinates:
  * index       (index) object MultiIndex
  * lat         (index) float64 -5.75 -5.5 -5.25 -5.0 ... 0.5 0.75 1.0 1.25
  * lon         (index) float64 52.0 52.0 52.0 52.0 52.0 ... 50.5 50.5 50.5 50.5
  * time        (index) object '2022-02-18 00:00:00' ... '2022-02-18 18:00:00'
Data variables:
    Unnamed: 0  (index) int64 0 1 2 3 4 5 ... 745411 745412 745413 745414 745415
    experiment  (index) object 'incr' 'incr' 'incr' ... 'curr' 'curr' 'curr'
    cfpf        (index) object 'cf' 'cf' 'cf' 'cf' 'cf' ... 'pf' 'pf' 'pf' 'pf'
    member      (index) int64 -1 -1 -1 -1 -1 -1 -1 -1 ... 49 49 49 49 49 49 49
    init        (index) object 'b2nr_2022-02-10' ... '1_2022-02-16'
    fg10        (index) float64 10.98 10.27 9.035 7.949 ... 30.55 31.58 31.41

In [ ]:
south_xr

# Identify extreme members

In [12]:
# plot pressures with cartopy
p = south_xr.isel(time=0).plot(
       subplot_kws=dict(projection=ccrs.PlateCarree(0), facecolor="gray"),
       transform=ccrs.PlateCarree(),
   )
    
p.axes.set_global()

p.axes.coastlines()

ValueError: Dimensions {'time'} do not exist. Expected one or more of Frozen({'index': 745416})